In [3]:


#erx = compile(r'((?:===+etymology(?: *[0-9] *)*===+ \n {{blend\|en)(\|[^}]+\}\}))')

from classes.AffixAnalyzer import setup
from classes.Word2 import Word
from classes.Lex2 import Lexicon
from classes.Chems import Chems
from utils.tools import lrsort, rrsort, cprint, load, dump, np, Counter, hprint
from lvsearch import lsearch
from re import compile

a = setup()
w = Lexicon(a.bare, True)

def load_hint() -> Lexicon:
    with open(f'C:\\Users\\BBA\\Coding\\NLP\\Embeddings\\data\\v0\\misc\\base7', 'rb') as f:
        return load(f)

def ssearch(word, corpus=None, mxln=50, disp=None, exc=None):
    if not corpus: corpus = w.media
    if not exc and not disp: return lrsort([x for x in corpus if x.startswith(word) and len(x) < mxln], trim=True)
    elif not exc: return lrsort([x for x in corpus if x[disp:].startswith(word) and len(x) < mxln], trim=True)
    elif isinstance(exc, (tuple, list, set)): return lrsort([x for x in corpus if x[(disp if disp else None):].startswith(word) and len(x) < mxln and all(not x.startswith(y) for y in exc)], trim=True)
    else: return lrsort([x for x in corpus if x[(disp if disp else None):].startswith(word) and len(x) < mxln and not x.startswith(exc)], trim=True)
def bsearch(word, corpus=None, mxln=50, disp=None, exc=None):
    if not corpus: corpus = w.media
    if not exc and not disp: return rrsort([x for x in corpus if x.endswith(word) and len(x) < mxln], trim=True)
    elif not exc: return rrsort([x for x in corpus if x[:-disp].endswith(word) and len(x) < mxln], trim=True)
    elif isinstance(exc, (tuple, list, set)): return rrsort([x for x in corpus if x[:(-disp if disp else None)].endswith(word) and len(x) < mxln and all(not x.endswith(y) for y in exc)], trim=True)
    else: return rrsort([x for x in corpus if x[:(-disp if disp else None)].endswith(word) and len(x) < mxln and not x.endswith(exc)], trim=True)

def syn_roots():
    tmp1, tmp2 = set(), set()
    for x in ('syn_1', 'syn_2', 'man_2', 'man_1'):
        tmp = [y.split() for y in w.get(x)]
        if x[0] == 's':
            for y in tmp:
                tmp1.add(y[0])
                for z in y[1:]:
                    if z[-1] == '+': tmp2.add(z[:-1])
        else:
            for y in tmp:
                tmp1.add(y[0])
    for x in tmp2:
        if any(x.endswith(y) for y in ('er', 'os', 'on', 'um', 'us', 'is', 'a')):
            if all(y not in x for y in ('_', '*', '~')):
                tmp1.add(x)
    return tmp1

def intermezzo():
    w['a'] = Word('a')
    w.supdate('a', ('an',))
    if 'a' in w.bases: w.bases.remove('a')
    if 'i' in w.bases: w.bases.remove('i')
    rmls = []
    for x in w.bases:
        if len(x) < 4:
            if x not in w.media: w.media[x] = Word(x)
            w.media[x].tid = 1
            rmls.append(x)
    for x in rmls: w.bases.remove(x)
    for x in w.get('uniqs'):
        if x in w.bases: w.bases.remove(x)
        if x not in w.media: print(f'Add {x}')
        else: w.media[x].exc = True
    print(len(w.bases))

def suforms(word, isl):
    attempts = []
    for sl in (isl, isl-1, isl+1):
        if not sl: continue
        if len(word) < sl + 2: continue
        attempts.extend([f'{word[:-sl]}{y}' for y in ('', 'us', 'um', 'is', 'o', 'e', 'a', 'y', 'os', 'on') if not y == word[-sl:] and f'{word[:-sl]}{y}' in a.bare])
    return attempts

def findx(piece, word):
    id, idx = 0, -1
    if piece[-1] == '_':
        piece = piece[:-1]
        while len(piece) - id > 1:
            if piece[id if id > 0 else None:] in word:
                idx = word.index(piece[id if id > 0 else None:])
                break
            else: id += 1
    else:
        if piece[0] == '_':
            maxl = 1
            piece = piece[1:]
        else: maxl = 2
        while len(piece) - id > maxl:
            if piece[:-id if id > 0 else None] in word:
                idx = word.index(piece[:-id if id > 0 else None])
                break
            else: id += 1
    return idx

def conv_cmls():
    ndct = {}
    keeper = {}
    for x in w.rgen['cmls']:
        if x[0] == 'S':
            if f'S{x[1]}' in keeper:
                print(keeper[f'S{x[1]}'], x)
            ndct[f'S{x[1]}'] = ' '.join(x)
            keeper[f'S{x[1]}'] = x
        else:
            hold = []
            placed = False
            ridx = findx(x[1], x[2])
            if ridx == 0:
                hold.append(x[1])
                placed = True
            for y in x[3:]:
                if placed: hold.append(y)
                else:
                    yidx = findx(y, x[2])
                    if ridx >= 0 and yidx >= 0:
                        if ridx < yidx:
                            hold.append(x[1])
                            hold.append(y)
                            placed = True
                        else: hold.append(y)
                    elif y[0] == '_':
                        hold.append(y)
                    elif y[-1] == '_':
                        hold.append(x[1])
                        hold.append(y)
                        placed = True
                    else:
                        if yidx >= 0:
                            if not hold and yidx > 0:
                                hold.append(x[1])
                                hold.append(y)
                                placed = True
                            else: hold.append(y)
                        else: hold.append(y)
            if not placed: hold.append(x[1])
            if x[2] in keeper:
                print(keeper[x[2]], x)
            ndct[x[2]] = ' '.join(hold)
            keeper[x[2]] = x
    return ndct

def chem2dct():
    c = Chems()
    c.dbreak(True)
    ndct = conv_cmls()
    cnt = Counter()
    tmp = 'DES GAL FES HIV PXN XA sarinu somanu tabunu phosgeneu decan nonadecan tetradecan octadecan pentadecan heptadecan hexadecan tridecan undecan dodecan ethan methan nonan propan icosan eicosan octan pentan heptan butan hexan ribos'.split()
    for x in c.cdct.items():
        if ' ' not in x[1]: continue
        for y in x[1].split():
            if y not in w.bt:
                if '_' in y: continue
                if y in tmp: continue
                cnt[y] += 1
    cafx = '_per _lev _dex an_ ane_ ate_ ene_ ic_ ide_ ine_ ite_ itol_ ium_ ose_ ous_ yl_ yne_'.split()
    for x in c.cdct.items():
        if ' ' not in x[1]: continue
        if x[0] in ndct: continue
        repack = [y if y not in cnt else '_uwid' for y in x[1].split()]
        repack = [y if y not in cafx else (f'{y}C' if y[0] == '_' else f'C{y}') for y in repack]
        print(f'M {x[0]} {" ".join(repack)}')
    for x in ndct.items():
        if x[0][0] == 'S':
            print(x[1])
        else:
            print(f'M {x[0]} {x[1]}')

def wkformer():
    for word in ewkd:
        hold = []
        for x in ewkd[word].split('\n'):
            if len(x) > 128:
                while len(x) > 128:
                    mark = 128
                    if any(z in x[:127] for z in '.,]}):#'):
                        while x[mark] not in '.,]}):#':
                            mark -= 1
                    elif any(z in x[:127] for z in '|= '):
                        while x[mark] not in '|= ':
                            mark -= 1
                    hold.append(x[:mark+1].strip())
                    x = x[mark+1:]
                hold.append(x.strip())
            else: hold.append(x.strip())
        ewkd[word] = '\n'.join(hold)
    for word in lwkd:
        hold = []
        for x in lwkd[word].split('\n'):
            if len(x) > 128:
                while len(x) > 128:
                    mark = 128
                    if any(z in x[:127] for z in '.,]}):#'):
                        while x[mark] not in '.,]}):#':
                            mark -= 1
                    elif any(z in x[:127] for z in '|= '):
                        while x[mark] not in '|= ':
                            mark -= 1
                    hold.append(x[:mark+1].strip())
                    x = x[mark+1:]
                hold.append(x.strip())
            else: hold.append(x.strip())
        lwkd[word] = '\n'.join(hold)

with open(f'C:\\Users\\BBA\\Coding\\NLP\\Embeddings\\data\\v0\\roots\\wikt', 'rb') as f: ewkd, lwkd = load(f)
with open(f'C:\\Users\\BBA\\Coding\\NLP\\Embeddings\\data\\v0\\roots\\memoir', 'rb') as f:
    memoir = load(f)
vowels = 'aeiouy'
a.assign_search_dict(a.bare)
for x in [x for x in w.get('_a1', a.bare) if x[1:] in a.bare]: w.mupdate(x, ('_an', x[1:]))
for x in [x for x in w.get('_a2', a.bare) if x[1:] in a.bare]: w.mupdate(x, ('_ad', x[1:]))
for x in [x for x in w.get('_e', a.bare) if x[1:] in a.bare]: w.mupdate(x, ('_e', x[1:]))

w.pl_parse()
w.prpt_parse()
w.pt_parse()
w.adjv_parse()
w.mbr_parse()
w.sfx1_parse()
w.en_parse()
#w.pre_parse()
intermezzo()
#w.ucodir(10)
w.tests()

exends = 'able ible ac aemia al and arch arium atid blast carp carpa cep ceps cide cyst cyte derm dom dont eal ean el ellar emia ern escens esis esque esthesia ette fera flora folia folius folium form gen gonia gram hedron ial ical id ifer il ile inal ine ion ique ism itis itude lite lith lysis morph nym oid oides oir oma openia opod optera orium osis philia phore phyll phylla phyte plasia plegia pnea pnoea pore sarcoma saur saurid some spore stasis tonia topia troph virus vore'.split()
latends = 'a aca acea aceus aceum acum acus aca ace ae alis ana ans antha ant anus anum ana ane ar aria arium arius aris arum as atis atum ata atus ate cea ea ella ens ensis e eo er eum eur eus ia ian ie iae ias iasis ibus icans icola icum icus ica ice id ida idia idis idium idum idus ide ienne ilis illa illo imus imum ima ime ina inus inum ine io is ismo issima issimo ista ite ito itus ium ius o on or ora orus orum os osa ose osum osus tia ula ular ule ulum ulus um us uum uus ax ix ex x y ys'.split()
cmb = rrsort({*exends, *latends, *w.sufx})

def latbreak(afx, twords, cidlim=5, mxdiff=3, rep=None, fails=False):
    latends = 'a aca acea aceus aceum acum acus aca ace ae alis ana ans antha ant anus anum ana ane ar aria arium arius aris arum as atis atum ata atus ate cea ea ella ens ensis e eo er eum eur eus ia ian ie iae ias iasis ibus icans icola icum icus ica ice id ida idia idis idium idum idus ide ienne ilis illa illo imus imum ima ime ina inus inum ine io is ismo issima issimo ista ite ito itus ium ius o on or ora orus orum os osa ose osum osus tia ula ular ule ulum ulus um us uum uus ax ix ex x y ys'.split()
    primes = 'us um os is a e on as o y'.split()
    preem = [x for x in [y.strip('+') for x in w.rgen['cmls'] for y in x[1:] if x[0] == 'S'] if '_' not in x and len(x) < 16 and any(x.endswith(z) for z in primes)]
    if not rep: rep = f'{afx}_'
    for x in twords:
        if x in w.media and x in w.bases and x.endswith(afx):
            i = len(afx)
            targ = x[:-i]
            checked = set()
            xbool = False
            hold = {}
            if f'{x[:-len(afx)]}' in w.media: hold[f'{x[:-len(afx)]}'] = f'M {x} {x[:-len(afx)]} {rep}'
            else:
                for p in primes:
                    if len(afx) > 2 and afx[0] in 'aeiou' and f'{x[:-(len(afx)-1)]}{p}' in w.media: hold[f'{x[:-(len(afx)-1)]}{p}'] = f'M {x} {x[:-(len(afx)-1)]}{p} {rep}'
                    elif f'{x[:-len(afx)]}{p}' in w.media: hold[f'{x[:-len(afx)]}{p}'] = f'M {x} {x[:-len(afx)]}{p} {rep}'
            if hold:
                if any(z in preem for z in hold): hold = {z: hold[z] for z in hold if z in preem}
                hold = {z: hold[z] for z in hold if not (z.endswith('o') and f'{z}s' in hold)}
                for z in hold: print(hold[z])
            else:
                while not xbool:
                    check = sorted(ssearch(targ, mxln=len(x)), key=lambda y: len(y))
                    for ci in check:
                        if ci in checked: continue
                        if ci != x and any(ci.endswith(z) for z in latends) and len(x) <= len(ci)+len(afx)+mxdiff:
                            print(f'M {x} {ci} {rep}')
                            xbool = True
                            break
                        else: checked.add(ci)
                    if xbool: break
                    if len(targ) < 4 or i-len(afx) > mxdiff or len(x)-i < cidlim:
                        if fails: print(f'M {x} _uwid {rep}')
                        break
                    else:
                        i += 1
                        targ = x[:-i]

def edgescan(low, high, side):
    cnt = Counter()
    for i in range(low, high):
        for x in w.bases:
            if len(x) / 2 >= i:
                if side == 'r': cnt[x[-i:]] += 1
                else: cnt[x[:i]] += 1
    for x in sorted(cnt, key=lambda z: 1/len(z)):
        for y in sorted(cnt, key=lambda z: len(z)):
            if len(y) <= len(x): continue
            if side == 'r':
                if y.endswith(x):
                    cnt[x] -= cnt[y]
                    break
            else:
                if y.startswith(x):
                    cnt[x] -= cnt[y]
                    break
    for x in cnt.most_common(): print(x[0], x[1])

def seper(word):
    wl = len(word)
    mid = wl // 2
    tail = wl // 4
    i = 0
    opts = []
    while mid-i > tail:
        if word[:mid+i] in w.media and word[mid+i:] in w.media:
            return (word[:mid+i], word[mid+i:])
        elif word[:mid-i] in w.media and word[mid-i:] in w.media:
            return (word[:mid-i], word[mid-i:])
        elif word[:mid+i] in w.media or word[mid+i:] in w.media:
            opts.append((word[:mid+i], word[mid+i:]))
        elif word[:mid-i] in w.media and word[mid-i:] in w.media:
            opts.append((word[:mid-i], word[mid-i:]))
        i += 1
    return opts

def rootfind(word):
    targs = ssearch(word, w.media)
    fls = set()
    if len(word) > 3 and word[-1] in 'aeiouy': targs.extend([x for x in ssearch(word[:-1], w.media) if len(x) < len(word)+2])
    targs = sorted(targs, key=lambda x: len(x))
    for x in targs[::-1]:
        if w[x].bcomp:
            for y in w[x].bcomp:
                if y in targs:
                    if x in targs: targs.remove(x)
                    fls.add(y)
    targs = [x for x in targs if x in fls]
    if len(targs) == 1: return targs[0]
    elif len(targs) > 1:
        targs = tuple([z for z in targs if len(z) <= len(targs[0])+1])
        if len(targs) == 1: return targs[0]
        else: return targs

def cleave(part, side='r', raf=False):
    targets = lrsort(ssearch(part, w.bases)) if side == 'l' else rrsort(bsearch(part, w.bases))
    if part in targets: targets.remove(part)
    if side == 'l':
        fr = rootfind(part)
        for x in targets:
            if x[len(part):] not in w.media and x[len(part):][0] in 'aiou' and x[len(part)+1:] in w.media: print(f'M {x} {fr} {x[len(part)+1:]}_' if raf else f'M {x} {fr} {x[len(part)+1:]}')
            else: print(f'M {x} {fr} {x[len(part):]}_' if raf else f'M {x} {fr} {x[len(part):]}')
    else:
        for x in targets:
            fronts = rootfind(x[:x.index(part)])
            if isinstance(fronts, str): print(f'M {x} {fronts} {x[x.index(part):]}_' if raf else f'M {x} {fronts} {x[x.index(part):]}')
            elif isinstance(fronts, tuple):
                for f in fronts: print(f'M {x} {f} {x[x.index(part):]}_' if raf else f'M {x} {f} {x[x.index(part):]}')
            else: print(f'M {x} {x[:x.index(part)]} {x[x.index(part):]}_' if raf else f'M {x} {x[:x.index(part)]} {x[x.index(part):]}')

def latex(word):
    out = set()
    ods = set()
    for ldct in (ewkd, lwkd):
        if word not in ldct: continue
        corpus = ldct[word]
        seps = ltx1.findall(corpus)
        for i, x in enumerate(seps):
            for pos in ('etymology', 'noun', 'verb', 'adverb', 'adjective', 'participle'):
                if pos not in x: continue
                langs = ('grc', 'ger', 'fr', 'esp', 'la', 'en') if pos == 'etymology' else ('grc', 'ger', 'fr', 'esp', 'la', 'grc-noun', 'grc-verb', 'grc-adjective', 'grc-adverb', 'en-noun', 'en-verb', 'en-adjective', 'en-adverb', 'la-noun', 'la-verb', 'la-adjective', 'la-adverb')
                for z in (y.split('|') for y in ltx2.findall(corpus[corpus.index(x) + len(x):corpus.index(seps[i+1]) if i < len(seps)-1 else None])):
                    for lang in langs:
                        if not (lang in z and lang != z[-1]): continue
                        for wp in z[z.index(lang)+1:]:
                            if len(wp) < 3 or wp == 'nl.' or '//' in wp or '-pro' in wp or wp[0] == '*': continue
                            if ' ' in wp: ods.add(wp)
                            else: out.add(wp)
                        break
    return out, ods

def sec_search(sec, tar, rep, filt=None):
    for x in w.media:
        if x not in ewkd: continue
        if filt and not x.endswith(filt): continue
        corpus = ewkd[x]
        seps = ltx1.findall(corpus)
        for i, y in enumerate(seps):
            if sec not in y: continue
            subg = corpus[corpus.index(y)+len(y):corpus.index(seps[i+1]) if i < len(seps)-1 else None]
            bracks = [z.split('|') for z in ltx2.findall(subg)]
            for z in bracks:
                if not tar in z or not f'-{tar}': continue
                print(f'M {x} {z[-1]} {rep}_')

def dshow(wrd):
    """ wcombos = [wrd[:-len(x)] for x in latends if wrd.endswith(x)]
    if not wrd[:-1] in wcombos: wcombos.append(wrd[:-1])
    wcombos = sorted(wcombos, key=lambda x: len(x))
    for x in w.rgen['cmls']:
        for y in wcombos:
            if any(z.startswith(y) for z in x[1:]):
                hprint(' '.join(x), wrd)
                break
    print('') """
    if wrd in ewkd: print(ewkd[wrd])
    print('<---------------->\nLAT')
    if wrd in lwkd: print(lwkd[wrd])
    """ print('')
    for x in ssearch(wcombos[-1], w.media): print(x) """
    if wrd in w.media:
        print(w[wrd].acomp)
        print(w[wrd].bcomp)

def onomos():
    for x in lrsort(w.media):
        if x in ewkd and 'nomatopoeic' in ewkd[x]:
            if x not in w.bases and w[x].bcomp: print(f'M {x} {" ".join(w[x].bcomp)}')
            else: print(f'M {x} _uwid onmp_')

def synsync():
    for x in w.rgen['cmls']:
        if x[0] != 'S': continue
        if not w[x[1]].bcomp: continue
        hold = []
        for y in x[2:]:
            w[y.strip('+')].bcomp = w[x[1]].bcomp
        for y in x[1:]:
            hold.extend(w[y.strip('+')].acomp)
        for y in x[1:]:
            w[y.strip('+')].acomp = [z for z in hold]

def findvsub(vgrp):
    hold = [x for x in w.media if vgrp in x and not x.startswith(vgrp) and not x.endswith(vgrp) and len(x) < 7 and '_' not in x and not x[0].isupper()]
    out = {}
    for x in sorted(hold, key=lambda x: len(x)):
        if w[x].bcomp and w[x].bcomp[0] in out: continue
        sgrp = x[:x.index(vgrp)]
        egrp = x[x.index(vgrp)+len(vgrp):]
        for i, y in enumerate(egrp):
            end = None
            if y in 'aeiouy':
                end = i
                break
        if end: egrp = egrp[:end]
        valt = []
        for y in ssearch(sgrp, mxln=7):
            if y == x: continue
            if '_' in y: continue
            if y[0].isupper(): continue
            if len(y) < len(sgrp)+len(vgrp)+1: continue
            if y[len(sgrp)] not in 'aeiou': continue
            for i, z in enumerate(y[len(sgrp):]):
                if z not in 'aeiou':
                    """ if y[len(sgrp)+i:].startswith(egrp):
                        valt.append((y, y[len(sgrp):len(sgrp)+i])) """
                    if y[len(sgrp)+i:] == egrp or y[len(sgrp)+i:] == f'{egrp}e':
                        valt.append((y, y[len(sgrp):len(sgrp)+i]))
                    break
        if valt:
            out[x] = tuple(valt)
    return out

def combine():
    uelems = set()
    ndct = {}
    for x in w.media:
        if w.media[x].bcomp:
            ndct[x] = list(w.media[x].bcomp)
        else:
            uelems.add(x)
    
    changed = True
    while changed:
        changed = False
        for x in ndct.items():
            xli = len(x[1])-1
            for i, y in enumerate(x[1][::-1]):
                if y not in ndct:
                    uelems.add(y)
                else:
                    changed = True
                    x[1].pop(xli-i)
                    for z in ndct[y][::-1]: x[1].insert(xli-i, z)
            ndct[x[0]] = x[1]
    return ndct, uelems

def secfind(sec):
    for x in lrsort(w.media):
        if x not in ewkd: continue
        corpus = ewkd[x]
        if sec not in corpus: continue
        seps = ltx1.findall(corpus)
        if any(sec in y for y in seps):
            hprint(x)
            for i, y in enumerate(seps):
                if sec not in y: continue
                subg = corpus[corpus.index(y)+len(y):corpus.index(seps[i+1]) if i < len(seps)-1 else None]
                print(y)
                print(subg)

def mwords():
    for x in sorted(ewkd, key=lambda x: (len(x), x)):
        if not 3 < len(x) < 30: continue
        if any(y not in 'abcdefghijklmnopqrstuvwxyz' for y in x): continue
        if x not in w.media: print(x)

wkformer()
synsync()
ltx1 = compile(r'(?:===+[a-z0-9 ]+===+)')
ltx2 = compile(r'\{\{([^|}{]+(?:\|[^|}{]*)+)}}')

ndct, uels = combine()
ndct = {x[0]: tuple(x[1]) for x in ndct.items()}
rdct = {" ".join(x[1]): x[0] for x in ndct.items()}
wpos = {}

cnt = Counter()
cnt2 = Counter()
for x in ndct.values():
    for i, y in enumerate(x):
        if '_' in y:
            cnt[y] += 1
        if i and y[-1] == '_' and x[i-1][-1] == '_':
            cnt2[f'{x[i-1]} {y}'] += 1
for x in w.media:
    if x not in ewkd: continue
    hold = []
    heads = ltx1.findall(ewkd[x])
    if '===noun===' in heads: hold.append('noun')
    if '===verb===' in heads: hold.append('verb')
    if '===adjective===' in heads: hold.append('adj')
    if '===adverb===' in heads: hold.append('adv')
    if hold: wpos[x] = tuple(hold)


85763 Words
43078 Words, 88928 Fragments
17950 items combined for plurals
24965 remaining
5276 items combined for present progressives
19689 remaining
5667 items combined for past 
14022 remaining
4300 items combined for adjectives 
9722 remaining
2665 items combined for membership 
7057 remaining
1663 items combined for sfx1 
5394 remaining
142 items combined for en suffix
5252 remaining
1832


In [ ]:
for x in lrsort(w.bases): print(x)

In [4]:
a.search('fuc')

['fuchsia',
 'fuchsias',
 'fuck',
 'fucked',
 'fucker',
 'fuckin',
 'fucking',
 'fucks',
 'fucose',
 'fucosyltransferase',
 'motherfucker',
 'motherfuckers']

In [ ]:
pnxs = ('O_', 'C_', 'P_', 'PD_')
vrxs = ('s_', 'ed_', 'ing_', 'y_', 'en_', 'er_', 'or_', 'tor_')
adxs = ('er_', 'est_', 'ly_', 'ness_')
nxs = ('s_', 'y_')

In [ ]:
w.rgen['cmls']

In [ ]:
uls = set()
for x in w.rgen['cmls']:
    if x[0] == 'S':
        for y in x[2:]:
            uls.add(y.strip('+'))
for x in w.get('uniqs'): uls.add(x)
#for x in w.get('propnouns'): uls.add(x)
used = set()
for x in w.media:
    for y in w[x].bcomp:
        used.add(y)

In [ ]:
names = w.get('propnouns')
for x in sorted(uls):
    if x[0].isupper(): continue
    if '_' in x: continue
    if x in a.bare: continue
    if x in used: continue
    if w[x].bcomp: continue
    if x in names: continue

    print(x)

In [ ]:
ssearch('quat', w.media)

In [ ]:
w['persicus']

In [ ]:
def step1():
    comms = ('_pre', 'auto', 'super', '_un', '_pro', '_mal', '_ex', '_per', '_ad', '_anti', 'trans', '_re', '_de', '_dis', '_mis', 'over', '_sub', '_co', 'in', '_in', "_an", '_tele', 'hypo', 'hyper', '_non', "_poly", '_tele', 'micro', 'mega', '_con', 'out')
    for x in ndct.items():
        if not x[0].endswith('tion'): continue
        if 'tion_' not in x[1] or 'ate_' not in x[1]: continue
        if x[1].index('ate_') != x[1].index('tion_') - 1: continue
        if " ".join(x[1][:x[1].index('ate_')+1]) in rdct: continue
        if x[1][0] in comms: continue
        print(x[0], x[1])
        for y in ndct:
            if y == x[0]: continue
            if x[0] in y:
                print(y, ndct[y])

def step2():
    for x in wpos.items():
        if 'adv' not in x[1]: continue
        if not (x[0].endswith('ly') and w[x[0]].bcomp and w[x[0]].bcomp[-1] != 'ly_' and w[x[0]].bcomp[-1] == 'y_'): continue
        for y in w.rgen['cmls']:
            if y[1] == x[0]:
                w.rcom((x[0], *w[x[0]].bcomp[:-1], 'ly_'))
                break
        else: w.rgen['cmls'].append(('M', x[0], *w[x[0]].bcomp[:-1], 'ly_'))


In [ ]:

""" for x in rrsort(cnt):
    if x[0] == '_': continue
    if x[0].isupper(): continue
    if x[0] == '~' or x[0] == '-': continue
    print(x, cnt[x]) """

for x in cnt.most_common():
    if x[0][0] == '_': continue
    if x[0][0].isupper(): continue
    if x[0][0] == '~' or x[0][0] == '-': continue
    print(x[0], x[1])

In [ ]:
lends = ('us_', 'um_', 'a_', 'o_', 'e_', 'ia_', 'ius_', 'ium_', 'io_', 'eus_', 'ea_', 'eo_', 'uus_')
for x in w.media:
    if not w[x].bcomp: continue
    if not w[x].bcomp[-1] in lends: continue
    for y in lends:
        if y == w[x].bcomp[-1]: continue
        if " ".join((*w[x].bcomp[:-1], y)) in rdct:
            print(x, rdct[" ".join((*w[x].bcomp[:-1], y))])


In [ ]:
for x in lrsort(w.media):
    if w[x].bcomp and '_' not in x: continue
    for y in ndct.items():
        if x in y[1]:
            break
    else:
        print(x)

In [ ]:
[x for x in w.rgen['cmls'] if 'ian_' in x]

In [ ]:
group = ('us', 'um', 'ium', 'ius', 'eus', 'eum', 'a', 'o', 'i', 'os', 'is', 'eo', 'io', 'e', 'ia', 'ea', 'ae', 'y', '')
sam = 'anus'
sal = len(sam)-2
for x in lrsort(rrsort(bsearch(sam))):
    if w[x].bcomp: print(f'{x} {" ".join(w[x].bcomp)}')
    else: print(x)
    ggr = lrsort(ssearch(f'{x[:-len(sam)]}'))
    if len(ggr) > 20 and len(f'{x[:-len(sam)]}') < 4: continue
    for y in lrsort(ssearch(f'{x[:-len(sam)]}')):
        print(f'  {y} {" ".join(w[y].bcomp)}')


In [ ]:
grp = ('in', 'inus')
ag = [x[:-1] for x in w.media if x[-1] == '_' and not any(y.isupper() for y in x) and (x.endswith(f'{grp[0]}_') or x.endswith(f'{grp[1][1:]}_') or x.startswith(grp[0]))]
sav = [x[:-len(grp[1])] for x in w.media if x.endswith(grp[1]) and len(x[:-len(grp[1])]) > 3]
sfg = 'aria ary arius'.split()
for x in lrsort(rrsort(w.media)):
    if len(x) < 4 or '_' in x or '~' in x: continue
    if w[x].bcomp and any('C' in z for z in w[x].bcomp): continue
    if x.endswith(grp[1]):
        print('M', x, " ".join(w[x].bcomp))
    elif any(x.startswith(y) for y in sav) and (len(x[:-len(grp[1])]) >  any(x.endswith(y) for y in ag)):
        print('M', x, " ".join(w[x].bcomp))
    #elif any(x.endswith(y) for y in sfg):
    #    print('M', x, " ".join(w[x].bcomp))
    else:
        for y in ag:
            if not (x.endswith(y) and any(f'{x[:-len(y)]}{z}' in w.media for z in ag if (z != y and f'{z}s' != y and f'{y}s' != z))): continue
            print('M', x, " ".join(w[x].bcomp))
            break


In [ ]:

ups = set(w.ups)
swords = w.synw
aws = w.get('b_add')

for x in lrsort(w.media):
    if w[x].acomp or w[x].bcomp: continue
    if x in ups: continue
    if x in swords: continue
    if '_' in x[0] and x[1:] in w.cut: continue
    print(x)


In [ ]:
cnt = Counter()
for x in w.media:
    if '_' not in x: continue
    if x[0] == '_' and f'{x[1:]}' in w.cut: continue
    cnt[x] += len(w[x].acomp)
for x in lrsort(cnt):
    if any(y.isupper() for y in x): continue
    if x[0] == '_': continue
    if x[0] == '~': continue
    print(x, cnt[x])
""" for x in cnt.most_common():
    if any(y.isupper() for y in x[0]): continue
    if x[0][0] == '_': continue
    if x[0][0] == '~': continue
    print(x[0], x[1]) """

In [ ]:

#Suffix Testing
igls, c0 = (), []
term = 'ad'
termx = f'{term}_'
al = len(term)
c1 = lrsort([x for x in w.bases if x.endswith(term) and len(x) > len(term)+1 and not w[x].tid == 4 and len(x) > 4])
mp = 3
cgrp = [list() for _ in range(mp+mp+1)]
comout = []
ucnt = Counter()
ffilt = False
bdg = {'i': '', 'o': '', 'a': ''}

for x in c1:
    tests1 = [
        (mx := w.rep_afx(x, al)),
        (mx := w.rep_afx(x, al, t='e')),
        (mx := w.rep_afx(x, al, tx='lag', bridges=bdg)),
        (mx := w.rep_afx(x, al, t='e', tx='lag', bridges=bdg)),
        (mx := w.rep_afx(x, al, t='dbl', tx='v', bridges=bdg)),
        (mx := w.rep_afx(x, al, tx='v', bridges=bdg)),
        (mx := w.rep_afx(x, al, t='e', tx='v', bridges=bdg)),
    ]
    hints = latex(x)
    hp = f'| {" ".join(hints[0]) if hints[0] else ""} | {" ".join(hints[1]) if hints[1] else ""}'
    scount = len([y for y in tests1 if y])
    if scount > 1: c0.append('&')
    elif scount == 1:
        c0.append('-')
        for i, win in enumerate(tests1):
            if win:
                ucnt[i] += 1
                break
    else: c0.append('X')
    com = False
    for i, y in enumerate(tests1):
        if y:
            if isinstance(y, str): cgrp[i].append(y)
            else: cgrp[i].append(y[0])
            if not com:
                #if i < mp: comout.append(('M', x, y, f'{term}'))
                #elif i == mp: comout.append(('M', x, y[0], f'_{y[1]}'))
                comout.append(('M', x, y, termx))
                #comout.append(('M', x, y, termx, hp) if len(hp) > 6 else ('M', x, y, termx))
                com = True
        else: cgrp[i].append('_')
    if not com:
        tot = sorted(ssearch(f'{x[:-(len(term)+1 if x[:-len(term)][-1] in 'iao' else len(term))]}_'), key=lambda p: len(p))
        if tot and len(tot[0]) <= len(x)-len(term)+2: comout.append(("M", x, tot[0], termx))
        else: comout.append(("M", x, '_uwid', termx))
        #if tot and len(tot[0]) <= len(x)-len(term)+2: comout.append(("M", x, tot[0], termx, hp) if len(hp) > 6 else ("M", x, tot[0], termx))
        #else: comout.append(("M", x, '_uwid', termx, hp) if len(hp) > 6 else ("M", x, '_uwid', termx))

print(f'\t\t\t\t\t{len([x for x in c0 if x == "X"])} / {len(c1)}')
cprint([len(c1) - x.count('_') for x in cgrp], [10, 14], col_width=4, halign='r', hlite=True)
cprint([ucnt[i] for i, x in enumerate(cgrp)], [10, 14], col_width=4, halign='r', hlite=True)
cprint([c0, c1, *cgrp], [1, 1, 10, 14], halign={0: 'r', 1: 'l', 2: 'r'}, col_width=4, hlite=['X', '-', '&'])
print('')
comout = [tuple([y for y in x if y != '_']) for x in comout]
comout = [x if not isinstance(x[-1], tuple) else tuple([y if not isinstance(y, tuple) else y[0] for y in x]) for x in comout]
coln = len(comout)
for x in comout: print(' '.join(x))


In [ ]:
cnt = Counter()
for x in cmb: cnt[x] += len(bsearch(x, w.bases))
cnt.most_common()
tx = 'esis'
txe = [x for x in latends if x.endswith(tx) and x != tx]
txw = sorted([x for x in w.bases if x.endswith(tx) and not any(x.endswith(z) for z in txe)])
latbreak(tx, txw, mxdiff=2, fails=True)